In [3]:
import linked_census as lc
import pandas as pd
from tqdm import tqdm

# Migration and population data from census

In [3]:
path_save = '/cluster/work/gess/coss/users/anmusso/Packages/Data/system_of_cities/data'

In [ ]:
for y in tqdm(lc.CensusYear):
    migrations = lc.get_intercity_migrations(census_year=y, industry=lc.Industry.ALL, cluster_level=lc.PlaceClusterLevel.l5)
    migrations.to_csv(f'{path_save}/intercity_migrations_{y}_{lc.Industry.ALL.value.lower()}.csv.gz', compression='gzip')

In [ ]:
populations = []
for y in tqdm(lc.CensusYear):
    population = lc.get_city_population(census_year=y, cluster_level=lc.PlaceClusterLevel.l5)
    population.rename(columns={'population': f'population_{y.value}'}, inplace=True)
    populations.append(population)
    
populations = pd.concat(populations, axis=1)
populations.fillna(0, inplace=True)

place_info = lc.data.place_data[[f'consistent_place_{lc.PlaceClusterLevel.l5.value}', f'consistent_place_name_{lc.PlaceClusterLevel.l5.value}', 'lat', 'lon']]
place_info.groupby(f'consistent_place_{lc.PlaceClusterLevel.l5.value}').agg({'lat': 'mean', 'lon': 'mean', f'consistent_place_name_{lc.PlaceClusterLevel.l5.value}': 'first'})
place_info.rename(columns={'consistent_place_name_{lc.PlaceClusterLevel.l5.value}': 'city_name'})

populations = populations.merge(place_info, left_index=True, right_index=True)
populations.to_csv(f'{path_save}/place_data_cluster_{lc.PlaceClusterLevel.l5.value}.csv')